In [1]:
import pandas as pd

In [2]:
gender_frame = pd.read_csv('./data/enron/01_got_gender.csv', index_col=0)

In [3]:
gender_frame.head()

,f_dir,m_id,m_date,m_from,m_to,m_cc,m_bcc,m_subj,mime_vers,cont_type,encode,x_from,x_to,x_cc,x_bcc,x_fold,x_orig,x_fname,m_body,gender
0,./data/maildir/allen-p/_sent_mail/1,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,NaN,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst\n\n,Here is our forecast\n\n,boy
1,./data/maildir/allen-p/_sent_mail/10,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,NaN,NaN,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst\n\n,Traveling to have a business meeting takes the...,boy
2,./data/maildir/allen-p/_sent_mail/100,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,NaN,NaN,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf\n\n,test successful. way to go!!!,boy
3,./data/maildir/allen-p/_sent_mail/1000,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,NaN,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf\n\n,"Randy,\n\n Can you send me a schedule of the s...",boy
4,./data/maildir/allen-p/_sent_mail/1001,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,NaN,NaN,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf\n\n,Let's shoot for Tuesday at 11:45.,boy


In [4]:
# get features (m_body), target (gender)
df = gender_frame[['m_body', 'gender']].copy()

In [5]:
df.head()

,m_body,gender
0,Here is our forecast\n\n,boy
1,Traveling to have a business meeting takes the...,boy
2,test successful. way to go!!!,boy
3,"Randy,\n\n Can you send me a schedule of the s...",boy
4,Let's shoot for Tuesday at 11:45.,boy


In [6]:
# return the largest email body for preprocessing setup
ts = df.m_body.max()
tss = df.loc[0:20, 'm_body']

In [7]:
tss

0                             Here is our forecast\n\n 
1     Traveling to have a business meeting takes the...
2                        test successful.  way to go!!!
3     Randy,\n\n Can you send me a schedule of the s...
4                   Let's shoot for Tuesday at 11:45.  
5     Greg,\n\n How about either next Tuesday or Thu...
6     Please cc the following distribution list with...
7                      any morning between 10 and 11:30
8     1. login:  pallen pw: ke9davis\n\n I don't thi...
9     ---------------------- Forwarded by Phillip K ...
10    Mr. Buckner,\n\n For delivered gas behind San ...
11    Lucy,\n\n Here are the rentrolls:\n\n\n\n Open...
12    ---------------------- Forwarded by Phillip K ...
13    ---------------------- Forwarded by Phillip K ...
14    Dave, \n\n Here are the names of the west desk...
15             Paula,\n\n 35 million is fine\n\nPhillip
16    ---------------------- Forwarded by Phillip K ...
17    Tim,\n\nmike grigsby is having problems wi

In [11]:
# import regex
import regex as re

# import lemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize # creates tokenized words
from nltk.corpus import stopwords


# instantiate stemmer object
stemmer = WordNetLemmatizer()

# catch list
results = []

def process_text(ts):
    ts = re.sub(r'\W', ' ', str(ts)) # remove characters that are not word characters
    ts = re.sub(r'_+', ' ', str(ts), flags=re.I) # removes the underscore character
    ts = re.sub(r'\s+[a-zA-Z]\s+|\^[a-zA-Z]\s+', ' ', str(ts)) # removes all single characters
    ts = re.sub(r'\d', ' ', str(ts)) # removes numbers
    ts = ts.lower()
    ts = re.sub(r'\s+', ' ', str(ts), flags=re.I) # reduces multiple spaces to single space
    
    # lemmatization
    words = word_tokenize(ts)
    ts = [stemmer.lemmatize(word) for word in words]
    ts = ' '.join(ts)
    
    return ts

In [17]:
%%time
df['p_body'] = df.m_body.apply(process_text)
processed_body = df.copy()
processed_body = processed_body.drop('m_body', axis=1)
processed_body.to_csv('./data/enron/02_processed_body.csv')

Wall time: 29min 45s
